In [183]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def generate_makefile_code(num_layers):
    makefile_lines = []    
    
    # Target for the main executable
    makefile_lines.append(f'conv{num_layers}: conv{num_layers}.o conv_tb{num_layers}.o')
    makefile_lines.append(f'\tclang conv{num_layers}.o conv_tb{num_layers}.o -o conv{num_layers}\n')
   
    makefile_lines.append(f'conv_tb{num_layers}.o: conv_tb{num_layers}.cpp conv_tb{num_layers}.h conv{num_layers}.h')
    makefile_lines.append(f'\tclang -c conv_tb{num_layers}.cpp -o conv_tb{num_layers}.o\n')

    makefile_lines.append(f'conv{num_layers}.o: conv{num_layers}.cpp conv{num_layers}.h')
    makefile_lines.append(f'\tclang -c conv{num_layers}.cpp -o conv{num_layers}.o\n')

    # Clean target
    makefile_lines.append('.PHONY: clean')
    makefile_lines.append('clean:')
    makefile_lines.append('\trm -rf ' + f'\tconv{num_layers}.o conv_tb{num_layers}.o conv{num_layers} ' + ' '.join([f'output{j}.bin' for j in range(1, num_layers + 1)]) + '\n')

    # Join code lines to form the final output
    generated_makefile = '\n'.join(makefile_lines)
    return generated_makefile


def generate_testbench_code(num_layers):
    code_lines = []

    # Add header includes
    code_lines.append('#include <cstdio>')
    code_lines.append('#include <cstdlib>')
    code_lines.append(f'\n#include "conv_tb{num_layers}.h"')  # Updated to use num_layers
    code_lines.append(f'#include "conv{num_layers}.h"\n')  # Updated to use num_layers

    # Start of the main function
    code_lines.append('int main(void) {')

    # Declare input, weights, and output arrays
    for i in range(num_layers):
        code_lines.append(f'    type_t *I{i + 1} = (type_t *) malloc(C{i + 1} * H{i + 1} * H{i + 1} * sizeof(type_t));')
        code_lines.append(f'    type_t *W{i + 1} = (type_t *) malloc(M{i + 1} * C{i + 1} * R{i + 1} * R{i + 1} * sizeof(type_t));')
        code_lines.append(f'    type_t *OL{i + 1} = (type_t *) calloc(M{i + 1} * E{i + 1} * E{i + 1}, sizeof(type_t));')

    # Random initialization of inputs
    code_lines.append('\n    srand(1);')
    for i in range(num_layers):
        code_lines.append(f'    for(unsigned j = 0; j < C{i + 1} * H{i + 1} * H{i + 1}; j++)')
        code_lines.append(f'        I{i + 1}[j] = rand() % RANDROOF;')

    # Random initialization of weights
    for i in range(num_layers):
        code_lines.append(f'    for(unsigned j = 0; j < M{i + 1} * C{i + 1} * R{i + 1} * R{i + 1}; j++)')
        code_lines.append(f'        W{i + 1}[j] = rand() % RANDROOF;')

    # Convolution function calls
    code_lines.append('\n    // Perform convolutions')
    for i in range(num_layers):
        code_lines.append(f'    conv{i + 1}(I{i + 1}, W{i + 1}, OL{i + 1});')

    # Print output
    code_lines.append('\n    // Output results')
    code_lines.append(f'    for(int j = 0; j < M{num_layers} * E{num_layers} * E{num_layers}; j++) {{')
    code_lines.append(f'        printf("%x ", OL{num_layers}[j]);')
    code_lines.append('    }\n')

    # Write output to files
    code_lines.append('    // Write output to files')
    for i in range(num_layers):
        code_lines.append(f'    FILE *opf{i + 1} = fopen("output{i + 1}.bin", "wb");')
        code_lines.append(f'    fwrite(OL{i + 1}, sizeof(type_t), M{i + 1} * E{i + 1} * E{i + 1}, opf{i + 1});')
        code_lines.append(f'    fclose(opf{i + 1});')

    # Free allocated memory
    code_lines.append('\n    // Free allocated memory')
    for i in range(num_layers):
        code_lines.append(f'    if(I{i + 1}) free(I{i + 1});')
        code_lines.append(f'    if(W{i + 1}) free(W{i + 1});')
        code_lines.append(f'    if(OL{i + 1}) free(OL{i + 1});')

    # End of main function
    code_lines.append('\n    return EXIT_SUCCESS;')
    code_lines.append('}')

    # Join code lines to form the final output
    generated_code = '\n'.join(code_lines)
    return generated_code


def generate_testbench_header_code(num_layers):
    code_lines = []

    # Add header includes
    code_lines.append('#ifndef CONV_TB_H')
    code_lines.append('#define CONV_TB_H')
    code_lines.append('#define RANDROOF 256')
    code_lines.append('#endif')
    
     # Join code lines to form the final output
    generated_code = '\n'.join(code_lines)
    return generated_code


# Function to generate the convolution implementation code with the specified number of layers
def generate_conv_code(num_layers):
    code = "#include \"conv{}.h\"\n\n".format(num_layers)  # Header file for the specific layer count

    # Generate convolution functions for each layer
    for i in range(1, num_layers + 1):
        code += "void conv{}(type_t I{}[C{} * H{} * H{}], type_t W{}[M{} * C{} * R{} * R{}], type_t O{}[M{} * E{} * E{}]) {{\n".format(
            i, i, i, i, i, i, i, i, i, i, i, i, i, i)
        code += "    // LOF\n"
        code += "    for(int m = 0; m < M{}; m++) {{\n".format(i)
        code += "        // LIF\n"
        code += "        for(int c = 0; c < C{}; c++) {{\n".format(i)
        code += "            // LSY\n"
        code += "            for(int y = 0; y < E{}; y++) {{\n".format(i)
        code += "                // LSX\n"
        code += "                for(int x = 0; x < E{}; x++) {{\n".format(i)
        code += "                    // LFY\n"
        code += "                    for(int k = 0; k < R{}; k++) {{\n".format(i)
        code += "                        // LFX\n"
        code += "                        for(int l = 0; l < R{}; l++) {{\n".format(i)
        code += "                            int h1 = y * S{} - P{} + k;\n".format(i, i)
        code += "                            int h2 = x * S{} - P{} + l;\n".format(i, i)
        code += "                            type_t i = (h1 < 0 || h1 >= H{} || h2 < 0 || h2 >= H{}) ? 0 : I{}[h2 + (h1 + (c * H{})) * H{}];\n".format(
            i, i, i, i, i)
        code += "                            O{}[x + (y + (m * E{})) * E{}] += i * W{}[l + (k + (c + (m * C{})) * R{}) * R{}];\n".format(
            i, i, i, i, i, i, i)
        code += "                        }\n"  # End LFX
        code += "                    }\n"  # End LFY
        code += "                }\n"  # End LSX
        code += "            }\n"  # End LSY
        code += "        }\n"  # End LIF
        code += "    }\n"  # End LOF
        code += "}\n\n"  # End of function

    return code

# Function to generate the convolution header code with the specified number of layers
def generate_convh_code(num_layers):
    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\n\ntypedef unsigned type_t;\n\n"

    # Generate code for each section of the configuration for each layer
    code += "// Number of input feature maps (N. channels in)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t C{} = 256;\n".format(i)
        
    code += "\n// Number of output feature maps (N. channels out)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t M{} = 256;\n".format(i)

    code += "\n// Input feature map size (H x H)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t H{} = 6;\n".format(i)

    code += "\n// Convolution kernel size (R x R)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t R{} = 5;\n".format(i)

    code += "\n// Convolution kernel stride\n"
    for i in range(1, num_layers + 1):
        code += "const size_t S{} = 1;\n".format(i)

    code += "\n// Output feature map size (E x E)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t E{} = 6;\n".format(i)

    code += "\n// Input feature map size with padding (F x F)\n"
    for i in range(1, num_layers + 1):
        code += "const size_t F{} = ((E{} * S{} + R{} - 1) < H{}) ? H{} : (E{} * S{} + R{} - 1);\n".format(i, i, i, i, i, i, i, i, i)

    code += "\n// Padding size\n"
    for i in range(1, num_layers + 1):
        code += "const size_t P{} = (F{} - H{}) / 2;\n".format(i, i, i)

    # Add convolution function declarations
    code += "\n"
    for i in range(1, num_layers + 1):
        code += "void conv{}(type_t I{}[C{} * H{} * H{}], type_t W{}[M{} * C{} * R{} * R{}], type_t O{}[M{} * E{} * E{}]);\n".format(
            i, i, i, i, i, i, i, i, i, i, i, i, i, i)

    code += "\n#endif"
    
    return code


# Create a slider for user input
layer_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,  # Adjust as needed
    step=1,
    description='NLayers:',
    continuous_update=False
)

# Create a button to generate code
generate_button = widgets.Button(
    description='Generate code',
    button_style='success'
)

# Set up the button's click event
def on_button_click(b):
    clear_output(wait=True)
    display(layer_slider, generate_button)
    
    
    generated_makefile = generate_makefile_code(layer_slider.value)
    # Save to MakefileX
    with open(f'./repo/padlogicNLayers/Makefile{layer_slider.value}', 'w') as makefile_file:
        makefile_file.write(generated_makefile)
    print(f'Makefile{layer_slider.value} has been created and saved.')
    
    generate_testbench = generate_testbench_code(layer_slider.value)
    # Save to testbench
    with open(f'./repo/padlogicNLayers/conv_tb{layer_slider.value}.cpp', 'w') as makefile_file:
        makefile_file.write(generate_testbench)
    print(f'conv_tb{layer_slider.value}.cpp has been created and saved.')
    
    generate_testbench_header = generate_testbench_header_code(layer_slider.value)
    # Save to MakefileX
    with open(f'./repo/padlogicNLayers/conv_tb{layer_slider.value}.h', 'w') as makefile_file:
        makefile_file.write(generate_testbench_header)
    print(f'conv_tb{layer_slider.value}.h has been created and saved.')
    
    generate_convh = generate_convh_code(layer_slider.value)
    # Save to conv header
    with open(f'./repo/padlogicNLayers/conv{layer_slider.value}.h', 'w') as makefile_file:
        makefile_file.write(generate_convh)
    print(f'conv{layer_slider.value}.h has been created and saved.')
    
    generate_conv = generate_conv_code(layer_slider.value)
    # Save to conv cpp
    with open(f'./repo/padlogicNLayers/conv{layer_slider.value}.cpp', 'w') as makefile_file:
        makefile_file.write(generate_conv)
    print(f'conv{layer_slider.value}.cpp has been created and saved.')

generate_button.on_click(on_button_click)

# Display the slider and button
display(layer_slider, generate_button)


IntSlider(value=5, continuous_update=False, description='NLayers:', max=10, min=1)

Button(button_style='success', description='Generate code', style=ButtonStyle())

Makefile5 has been created and saved.
conv_tb5.cpp has been created and saved.
conv_tb5.h has been created and saved.
conv5.h has been created and saved.
conv5.cpp has been created and saved.


In [189]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
from git import Repo

# Function to clone a GitHub repository if not exist
def clone_repo(github_url, dest_dir):
    # Check if the destination directory exists
    if os.path.exists(dest_dir):
        # Check if the directory is a Git repository
        if os.path.exists(os.path.join(dest_dir, ".git")):
            print(f"Repository already cloned in {dest_dir}.")
            return  # Exit the function if the repository is already cloned
        else:
            print(f"Directory {dest_dir} exists but is not a Git repository.")
            # Optionally, you could choose to remove the existing directory or proceed differently
            return
    
    # Create the directory if it doesn't exist and clone the repository
    os.makedirs(dest_dir)
    Repo.clone_from(github_url, dest_dir)
    print(f"Cloned repository to {dest_dir}")

# Function to read constants and comments from .h file
def read_constants(file_path):
    constants = {}
    comments = {}
    current_comment = ""
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line = line.strip()
            # Collect comment lines
            if line.startswith("//"):
                current_comment = line  # Update to last comment line
            elif line.startswith("const size_t"):
                parts = line.split()
                constant_name = parts[2]
                try:
                    constant_value = int(parts[4].strip(";"))
                    constants[constant_name] = constant_value
                    # Store the last comment line
                    comments[constant_name] = current_comment.strip()  # Last line of comment
                    current_comment = ""  # Reset comment for the next constant
                except ValueError:
                    # Skip lines that don't contain simple integer values
                    continue
    return constants, comments

# Function to modify .h file with new values for constants
def modify_conv_constants(file_path, constants):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    new_lines = []
    for line in lines:
        if line.startswith("const size_t"):
            parts = line.split()
            constant_name = parts[2]
            if constant_name in constants:
                new_lines.append(f"const size_t {constant_name} = {constants[constant_name]};\n")
            else:
                new_lines.append(line)
        else:
            new_lines.append(line)
    
    with open(file_path, 'w') as file:
        file.writelines(new_lines)
    print(f"Updated constants in {file_path}")

# Function to run a command (Makefile or executable)
def run_command(destination_directory, command):
    result = subprocess.run(command, cwd=destination_directory, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    if result.returncode == 0:
        print("Command executed successfully!")
        print(result.stdout)
    else:
        print("Error executing command:")
        print(result.stderr)

# Function to display widgets for constant modification along with comments
def display_constant_widgets(constants, comments):
    widgets_list = []
    for name, value in constants.items():
        comment = comments.get(name, "")  # Get the associated last-line comment (if any)
        comment_label = widgets.HTML(value=f"<b>{comment}</b>")  # Display comment as HTML label
        constant_widget = widgets.IntText(value=value, description=name, layout=widgets.Layout(width='150px'))
        
        widgets_list.append(comment_label)
        widgets_list.append(constant_widget)
    
    # Create a button to save the changes
    save_button = widgets.Button(description="Update code & make")
    
    def on_save_button_click(b):
        # Update constants with new values from widgets
        for widget in widgets_list:
            if isinstance(widget, widgets.IntText):
                constants[widget.description] = widget.value
        modify_conv_constants(configure_path, constants)
        selected_makefile = 'Makefile' if file_dropdown.value == 'conv' else 'Makefile5'
        run_command(command_path, ['make', '-f', selected_makefile])
    
    save_button.on_click(on_save_button_click)
    
    # Display the widgets and button
    display(widgets.VBox(widgets_list + [save_button]))

# Function to display buttons
def display_buttons():
    display(run_button, reset_button)

# Clone the GitHub repository
github_url = "https://github.com/vbonato/paunajacaTestBench.git"
destination_directory = "./repo"
clone_repo(github_url, destination_directory)

# Set initial paths based on selected file
def update_paths_and_constants():
    global configure_path, constants, comments
    configure_path = os.path.join(destination_directory, "padlogicNLayers", f"{file_dropdown.value}.h")
    constants, comments = read_constants(configure_path)
    clear_output(wait=True)  # Clear output to refresh displayed widgets
    display(file_dropdown)  # Re-display the dropdown
    display_constant_widgets(constants, comments)  # Display updated widgets
    display_buttons()  # Re-display buttons

# Dropdown to choose between conv and conv5
file_dropdown = widgets.Dropdown(
    options=['conv', 'conv5'],
    value='conv5',
    description='Layers:',
    layout=widgets.Layout(width='180px')
)
file_dropdown.observe(lambda change: update_paths_and_constants(), names='value')

# Initial configuration path setup
configure_path = os.path.join(destination_directory, "padlogicNLayers", f"{file_dropdown.value}.h")
constants, comments = read_constants(configure_path)

# Create a button to run the code
run_button = widgets.Button(description="Run as x86")
reset_button = widgets.Button(description="Clear Display")

# Function to run the code
def run_code_click(b):
    executable = './conv' if file_dropdown.value == 'conv' else './conv5'
    run_command(command_path, [executable])

# Function to reset the output
def reset_output(b):
    clear_output(wait=True)  # Clear all output
    display(file_dropdown)  # Display the dropdown to choose between conv and conv5
    display_constant_widgets(constants, comments)  # Re-display constant widgets and comments
    display_buttons()  # Re-display run/reset buttons

# Set button clicks
run_button.on_click(run_code_click)
reset_button.on_click(reset_output)

# Initial display
display(file_dropdown)  # Display the dropdown to choose between conv and conv5
display_constant_widgets(constants, comments)
display_buttons()


Dropdown(description='Layers:', index=1, layout=Layout(width='180px'), options=('conv', 'conv5'), value='conv5…

Button(description='Run as x86', style=ButtonStyle())

Button(description='Clear Display', style=ButtonStyle())

Updated constants in ./repo/padlogicNLayers/conv5.h
Command executed successfully!
clang -c conv5.cpp -o conv5.o
clang -c conv_tb5.cpp -o conv_tb5.o
clang conv5.o conv_tb5.o -o conv5

Command executed successfully!
2478ed6 306b57e 3be7170 3b3cf22 2fb57f9 236f201 2fe4273 3fa6ab6 4fafc0f 4fb4af5 3f34892 2fe2363 3c15490 4fb06b7 63ca833 635cb39 4fae805 3bd9538 3cda189 503fc1a 64576fe 6496ad2 501cf5e 3beedfc 307d923 4003faa 50290ca 5026e31 40057ca 30f206f 24bec39 30fa1ed 3c76301 3c6ad66 2ff3b48 24ce3b1 24c0a8d 2fc5514 3bade15 3afc2d0 2e3d24e 226b08f 302e53c 3fd4e6c 4ec9df9 4d68714 3e67cbb 2ed19ad 3c08a10 4f69a73 622118e 61b932f 4d643dc 3a4e20d 3b5e0e1 4e7b2b4 621051d 61fb04b 4d1a35b 3a98a16 2fc220f 3e992cf 4dfc7f8 4e8da2a 3df7cdd 2e643ec 2367447 2ed48ea 3b20667 3b11eb0 2fe3e10 2360ab3 23a185b 2ec73b8 3b3b706 3a88219 2e65b2e 22ea300 2f3d0f9 3e0f686 4ebd80c 4e4bd78 3dfbecb 2e3f113 3b54d44 4e53620 6239d58 61d4e4b 4dc16f6 3ad2cec 3b09ff6 4daf315 628a1e3 6273619 4e38a5b 3aa631b 2eb7543 3e30500 4